<a href="https://colab.research.google.com/github/gourab-sinha/Machine_Learning/blob/master/Naive%20Bayes%20Classifier/Project/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTE: Run the script on Google Colab
###### LINK: https://colab.research.google.com/

In [0]:
# Load Packages
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import operator
import matplotlib.pyplot as plt
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Fetch words from 20 different categories
#### Categories are labelled with integer values


In [0]:
class TextClassification:

  # Categories with integer labels
  categories = {0:"alt_atheism",1:"comp_graphics",2:"comp_os_ms_windows_misc",3:"comp_sys_ibm_pc_hardware",
              4:"comp_sys_mac_hardware",5:"comp_windows_x",6:"misc_forsale",7:"rec_autos",
              8:"rec_motorcycles",9:"rec_sport_baseball",10:"rec_sport_hockey",11:"sci_crypt",
              12:"sci_electronics",13:"sci_med",14:"sci_space",15:"soc_religion_christian",
              16:"talk_politics_guns",17:"talk_politics_mideast",18:"talk_politics_misc",
              19:"talk_religion_misc"}

  # Stop words
  stop_words = set(stopwords.words('english'))

  # Function to get dictionary in the form of list(key-value pair for each element in list)
  def get_word_dictionary(self):

    # Store words with occurrence 
    dictionary = {}

    # Base Path 
    base_path = "/content/drive/My Drive/Colab Notebooks/Text Classifier Dataset/"

    # Iterate for each category
    for i in range(20):
      # File location
      path = base_path+str(categories[i])+".txt"
      with open(path, encoding="utf8", errors='ignore') as file_obj:
        file_data =  file_obj.readlines()
      
      # Iterate over each line and pick words except stop words
      for i in range(len(file_data)):
        for word in file_data[i].split():
          if word not in stop_words:
            dictionary[word.lower()] = dictionary.get(word.lower(),0)+1
    
    # Special and digits
    ignore_words_with_specialcharacters_digits = set(punctuation)
    for i in range(0,10):
      ignore_words_with_specialcharacters_digits.add(str(i))
    ignore_words_with_specialcharacters_digits

    # Remove all keys with special characters and digits
    for key in dictionary.copy():
      for char in ignore_words_with_specialcharacters_digits:
        if char in key:
          dictionary.pop(key)
          break
    
    # Sort based on occurrence
    sorted_dictionary = dict(sorted(dictionary.items(), key=operator.itemgetter(1),reverse=True))
    words = [key for key,val in sorted_dictionary.items()]
    value = [val for key,val in sorted_dictionary.items()]

    # Pick top 2000 most appeared words with their count
    top_2000_words = []
    i = 0
    for key,val in sorted_dictionary.items():
      top_2000_words.append([key,val])
      i+=1
      if i==2000:
        break
        
    return top_2000_words

    
    

In [0]:
clf = TextClassification()
top_2000_words = clf.get_word_dictionary()

In [312]:
top_2000_words

[['i', 65842],
 ['the', 25653],
 ['apr', 19602],
 ['gmt', 17684],
 ['would', 15512],
 ['in', 14773],
 ['one', 12976],
 ['article', 12108],
 ['like', 9434],
 ['if', 8564],
 ['people', 8415],
 ['get', 8331],
 ['university', 8203],
 ['it', 7806],
 ['know', 7695],
 ['think', 7205],
 ['may', 7126],
 ['this', 7018],
 ['use', 6251],
 ['also', 6166],
 ['x', 6152],
 ['new', 6086],
 ['could', 5905],
 ['even', 5400],
 ['good', 5278],
 ['a', 4961],
 ['make', 4916],
 ['many', 4872],
 ['and', 4812],
 ['you', 4810],
 ['see', 4691],
 ['two', 4557],
 ['much', 4530],
 ['time', 4336],
 ['first', 4114],
 ['want', 4104],
 ['say', 4051],
 ['what', 4034],
 ['anyone', 3976],
 ['need', 3918],
 ['but', 3891],
 ['way', 3791],
 ['they', 3788],
 ['us', 3745],
 ['used', 3742],
 ['go', 3617],
 ['world', 3602],
 ['really', 3507],
 ['since', 3418],
 ['right', 3326],
 ['believe', 3309],
 ['for', 3303],
 ['still', 3290],
 ['he', 3217],
 ['going', 3216],
 ['there', 3200],
 ['something', 3190],
 ['computer', 3157],
 ['fin